In [2]:
from common import *
from numba import jit
from heapq import *
nan32 = (np.array(0.0, dtype=np.float32) + np.nan).astype(np.float32)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


### high level redesign

In [2]:
class TheAlg():
    def __init__(a, X, Y, W, s, mvalid, algorithm, alltrain=False):
        a.alltrain = alltrain or not s.ho.sum()
        a._alg_evaluate = a._get_algorithm(algorithm, alltrain=alltrain)
        a._alg_finalize = a._get_algorithm(algorithm, finalize=True, alltrain=alltrain)
        a.s = s
        a.mvalid = mvalid.values
        a.k = X.columns.get_level_values(0).nunique() # features
        a.m = X.columns.get_level_values(1).nunique() # stocks/groups
        a.columns = X.columns
        a._def_matrices(X, Y, W, s) # Y not demeaned yet to get the no mean baseline score
        a._setup()
        a._args = ()
        
    def _setup(a):
        a.scores = {}
        a.scores[None] = a._calc_baseline()
        a.scores[None] = (a.scores[None][0], 1) if a.scores[None][1] == 0 else a.scores[None]
        a.meandata = (a.Y.tc * a.W.tc).sum(axis=0) / _replace_0_1(a.W.tc.sum(axis=0))
        a.meandatafn = (a.Y.tc * a.W.tc).sum(axis=0) + (a.Y.ho * a.W.ho).sum(axis=0)
        a.meandatafn /= _replace_0_1(a.W.tc.sum(axis=0) + a.W.ho.sum(axis=0))
        for sp in a.Y: # loop to demean Y
            a.Y[sp] -= a.meandata
        a.vardata = (a.Y.tc ** 2 * a.W.tc).sum(axis=0)
        a.scores[()] = a._calc_baseline()
        a.fbnums = {None: None, (): None}
        a.over = set()
        
    def _def_matrices(a, X, Y, W, s):
        a.X, a.Y, a.W = O(), O(), O()
        for aZ, Z in zip([a.X, a.Y, a.W], [X, Y, W]):
            aZ.tc = Z[s.tc].values
            aZ.d0 = np.stack([Z[tr].values for tr, cv in s.trcv])
            aZ.d1 = np.stack([Z[cv].values for tr, cv in s.trcv])
            aZ.ho = Z[s.ho].values
            aZ.aw = Z.values
        
    def _calc_baseline(a):
        return (sum(_lr_rss(y, np.zeros_like(y), w) for y, w in zip(a.Y.tc, a.W.tc)),
                sum(_lr_rss(y, np.zeros_like(y), w) for y, w in zip(a.Y.ho, a.W.ho)))
    
    def _get_algorithm(a, name, finalize=False, alltrain=False):
        if finalize:
            alg = 'finalize'
        elif alltrain:
            alg = 'alltrain'
        else:
            alg = 'evaluate'
        return globals()['_alg_' + alg + '_' + name]
            
    def alg1(a, start=(), branch=1, branch_at_depth=None, max_depth=4, print=print, dot=1, numdot=10):
        a._alg1 = O(branch=branch, branch_at_depth=branch_at_depth, max_depth=max_depth,
                    print=print, dot=dot, numdot=numdot)
        a._alg1_recur(start, depth=0)
        
    def _alg1_recur(a, feats, depth):
        feats, o, print = feats, a._alg1, a._alg1.print
        print((a.scores[feats][0] / a.scores[None][0], a.scores[feats][1] / a.scores[None][1]), a.fbnums[feats], len(feats))
        print('[' + ', '.join(str(a.columns[a.m * f][0]) for f in feats) + '] ' + str(feats))
        print(flush=True)
        if depth >= o.max_depth:
            return
        print('<{}>'.format(a.k - len(feats)), end=' ', flush=True)
        heap = []
        for i, f in enumerate(set(range(a.k)) - set(feats)):
            print('.' if i // o.dot % o.numdot else i, end='', flush=True) if i % o.dot == 0 else None
            newfeats = tuple(sorted(feats + (f,)))
            if newfeats not in a.scores:
                newscore, newfbnum = a._alg_evaluate(a.X.d0, a.Y.d0, a.W.d0,
                                                     a.X.d1, a.Y.d1, a.W.d1,
                                                     a.X.tc, a.Y.tc, a.W.tc,
                                                     a.X.ho, a.Y.ho, a.W.ho,
                                                     a.m, a.mvalid, a.vardata,
                                                     a.k, np.array(newfeats, dtype=np.int64), a.s.n_folds, *a._args)
                a.scores[newfeats] = newscore
                a.fbnums[newfeats] = newfbnum
            if a.scores[newfeats][0] < a.scores[feats][0]:
                heap.append((a.scores[newfeats][0], newfeats))
            else:
                a.over.add(newfeats)
        print('\n', flush=True)
        heap.sort()
        for score0, f in heap[:o.branch_at_depth(depth) if o.branch_at_depth else o.branch]:
            a._alg1_recur(f, depth + 1)
        
    def alg2(a, start=None, top=None, score=None, tight=False, print=print, dot=1, numdot=10):
        o = O(dot=dot, numdot=numdot)
        assert (top is not None) + (score is not None) + (start is not None) == 1
        start = a.scores if start is None else start
        heap = sorted((a.scores[f][0], f) for f in start if f not in a.over)
        score = float('inf') if score is None else score
        searchlim = heap[top][0] if top is not None else score
        while heap and heap[0][0] < searchlim:
            if tight:
                searchlim = heap[0][0]
            feats = heappop(heap)[1]
            print((a.scores[feats][0] / a.scores[None][0], a.scores[feats][1] / a.scores[None][1]),
                  a.fbnums[feats], len(feats))
            print('[' + ', '.join(str(a.columns[a.m * f][0]) for f in feats) + '] ' + str(feats))
            print(flush=True)
            print('<{}>'.format(a.k - len(feats)), end=' ', flush=True)
            for i, f in enumerate(set(range(a.k)) - set(feats)):
                print('.' if i // o.dot % o.numdot else i, end='', flush=True) if i % o.dot == 0 else None
                newfeats = feats + (f,)
                if newfeats not in a.scores:
                    newscore, newfbnum = a._alg_evaluate(a.X.d0, a.Y.d0, a.W.d0,
                                                         a.X.d1, a.Y.d1, a.W.d1,
                                                         a.X.tc, a.Y.tc, a.W.tc,
                                                         a.X.ho, a.Y.ho, a.W.ho,
                                                         a.m, a.mvalid, a.vardata,
                                                         a.k, np.array(newfeats, dtype=np.int64), a.s.n_folds, *a._args)
                    a.scores[newfeats] = newscore
                    a.fbnums[newfeats] = newfbnum
                    if a.scores[newfeats][0] < a.scores[feats][0]:
                        heappush(heap, (a.scores[newfeats][0], newfeats))
                    else:
                        a.over.add(newfeats)
            print('\n', flush=True)
            
    def final(a, feats_list, args_list, weight_list=None, dry=False, dot=1, numdot=10, average=True):
        print('<{}>'.format(len(feats_list)), end=' ', flush=True)
        weight_list = [1 for i in range(len(args_list))] if weight_list is None else weight_list
        Y_ = []
        for i, (feats, args, weight) in enumerate(zip(feats_list, args_list, weight_list)):
            print('.' if i // dot % numdot else i, end='', flush=True) if i % dot == 0 else None
            if dry:
                md, y_ = a._alg_finalize(a.X.tc, a.Y.tc, a.W.tc, a.X.aw,
                                         a.m, a.mvalid, a.k, np.array(feats, dtype=np.int64), *args)
            elif a.alltrain:
                md, y_ = a._alg_finalize(a.X.tc, a.Y.tc, a.W.tc, a.X.aw,
                                         a.m, a.mvalid, a.k, np.array(feats, dtype=np.int64), *args)
            else:
                md, y_ = a._alg_finalize(np.concatenate([a.X.tc, a.X.ho]),
                                         np.concatenate([a.Y.tc, a.Y.ho]),
                                         np.concatenate([a.W.tc, a.W.ho]), a.X.aw,
                                         a.m, a.mvalid, a.k, np.array(feats, dtype=np.int64), *args)
            Y_.append(weight * y_ if average else y_)
        print(flush=True)
        if average:
            return np.stack(Y_).sum(axis=0) / sum(weight_list) + (a.meandata if dry else a.meandatafn)
        else:
            return [y_ + (a.meandata if dry else a.meandatafn) for y_ in Y_]

In [3]:
def _replace_0_1(a):
    a[a == 0] = 1
    return a

In [ ]:
@jit(nopython=True) #doesn't work / overfits
def _alg_evaluate_super(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, mvalid, mbase, k, subk, f):
    tc = np.empty(m, dtype=Yt.dtype) + nan32
    ho = np.empty(m, dtype=Yh.dtype) + nan32
    tck = np.empty((m, subk.shape[0] + 1), dtype=Yt.dtype) + nan32
    hok = np.empty((m, subk.shape[0] + 1), dtype=Yh.dtype) + nan32
    #model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    repeatf = X0.shape[0]
    fbn = 0
    fulln = 0
    for g in range(m):
        tck[g, 0] = mbase[g] * (repeatf // f)
        hok[g, 0] = _lr_rss(Yh[:, g], 0, Wh[:, g])
        for j in range(1, subk.shape[0] + 1):
            head = subk[:j]
            X0g, Y0g, W0g, X1g, Y1g, W1g, xt, yt, wt, xh, yh, wh = (
                X0[:, :, g::m][:, :, head], Y0[:, :, g], W0[:, :, g],
                X1[:, :, g::m][:, :, head], Y1[:, :, g], W1[:, :, g],
                Xt[:, g::m][:, head], Yt[:, g], Wt[:, g],
                Xh[:, g::m][:, head], Yh[:, g], Wh[:, g])
            tcg = np.empty(repeatf, dtype=Y0g.dtype) + nan32
            for i in range(repeatf):
                x0, y0, w0, x1, y1, w1 = X0g[i], Y0g[i], W0g[i], X1g[i], Y1g[i], W1g[i]
                beta = _lr_fit(x0, y0, w0)
                y1_ = _lr_predict(x1, beta)
                tcg[i] = _lr_rss(y1, y1_, w1)
            tcg.sort()
            tck[g, j] = tcg.sum()
            beta = _lr_fit(xt, yt, wt)
            yh_ = _lr_predict(xh, beta)
            hok[g, j] = _lr_rss(yh, yh_, wh)
    for j in range(subk.shape[0] + 1):
        pass
    tc.sort()
    ho.sort()
    return (tc.sum() / (repeatf // f), ho.sum()), (fbn, m - fulln)

In [ ]:
@jit(nopython=True)
def _exp_weigh(x, a):
    return x * (1 - np.exp(-a))

@jit(nopython=True)
def _alg_evaluate_expfb(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, mvalid, mbase, k, subk, f, _val, _exp):
    Mval, Mexp = len(_val), len(_exp)
    tc = np.zeros((m, Mval, Mexp), dtype=Yt.dtype) # zeros because we need to add to this for each fold
    ho = np.empty((m, Mval, Mexp), dtype=Yh.dtype) + nan32 # nans because we only set each value once for the HO set
    #model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    repeatf = X0.shape[0]
    for g in range(m): # loop for each stock
        X0g, Y0g, W0g, X1g, Y1g, W1g, xt, yt, wt, xh, yh, wh = (
            X0[:, :, g::m][:, :, subk], Y0[:, :, g], W0[:, :, g],
            X1[:, :, g::m][:, :, subk], Y1[:, :, g], W1[:, :, g],
            Xt[:, g::m][:, subk], Yt[:, g], Wt[:, g],
            Xh[:, g::m][:, subk], Yh[:, g], Wh[:, g])
        ############## TRCV dataset
        for p in range(repeatf): # loop for each cv-fold
            x0, y0, w0, x1, y1, w1 = X0g[p], Y0g[p], W0g[p], X1g[p], Y1g[p], W1g[p]
            beta = _lr_fit(x0, y0, w0)
            y1_ = _lr_predict(x1, beta)
            # search over the exp fallback
            thei = Mval
            for i in range(Mval): # i + 1 = min num of obs in stock - ddof to use lin model
                if _val[i] >= mvalid[g] - subk.shape[0]:
                    thei = i
                    break
                tc[g, i, 0] += _lr_rss(y1, y1_, w1) # when j == 0 just use the linear model
                for j in range(1, Mexp): # j denotes exponential weighting argument
                    y1_fb = _exp_weigh(y1_, (mvalid[g] - subk.shape[0] - _val[i]) / _exp[j])
                    tc[g, i, j] += _lr_rss(y1, y1_fb, w1)
        # outside the cv-fold loop
        tc[g] /= repeatf // f # divide by number of times trcv set was repeated over
        tc[g, thei:] = mbase[g] # fill in the mean model prediction errors for when mvalid[g] < i
        ############# HO dataset
        beta = _lr_fit(xt, yt, wt)
        #model[g] = beta
        yh_ = _lr_predict(xh, beta)
        # search over the exp fallback
        thei = Mval
        for i in range(Mval): # i + 1 = min num of obs in stock - ddof to use lin model
            if _val[i] >= mvalid[g] - subk.shape[0]:
                thei = i
                break
            ho[g, i, 0] = _lr_rss(yh, yh_, wh) # when j == 0 just use the linear model
            for j in range(1, Mexp): # j denotes exponential weighting argument
                yh_fb = _exp_weigh(yh_, (mvalid[g] - subk.shape[0] - _val[i]) / _exp[j])
                ho[g, i, j] = _lr_rss(yh, yh_fb, wh)
        ho[g, thei:] = _lr_rss(yh, 0, wh) # fill in the mean model prediction errors for when mvalid[g] < i
        #%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%
    tcfb, hofb = tc.sum(axis=0), ho.sum(axis=0)
    hoij, hobest = hofb.argmin(), hofb.min()
    tcij, tcbest = tcfb.argmin(), tcfb.min()
    return (tcbest, hobest), ((_val[tcij // Mexp], _exp[tcij % Mexp]), (_val[hoij // Mexp], _exp[hoij % Mexp]))

In [ ]:
@jit(nopython=True)
def _alg_evaluate_fb(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, mvalid, mbase, k, subk, f):
    tc = np.empty(m, dtype=Yt.dtype) + nan32
    ho = np.empty(m, dtype=Yh.dtype) + nan32
    #model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    repeatf = X0.shape[0]
    for g in range(m):
        X0g, Y0g, W0g, X1g, Y1g, W1g, xt, yt, wt, xh, yh, wh = (
            X0[:, :, g::m][:, :, subk], Y0[:, :, g], W0[:, :, g],
            X1[:, :, g::m][:, :, subk], Y1[:, :, g], W1[:, :, g],
            Xt[:, g::m][:, subk], Yt[:, g], Wt[:, g],
            Xh[:, g::m][:, subk], Yh[:, g], Wh[:, g])
        tcg = np.empty(repeatf, dtype=Y0g.dtype) + nan32
        for i in range(repeatf):
            x0, y0, w0, x1, y1, w1 = X0g[i], Y0g[i], W0g[i], X1g[i], Y1g[i], W1g[i]
            beta = _lr_fit(x0, y0, w0)
            y1_ = _lr_predict(x1, beta)
            tcg[i] = _lr_rss(y1, y1_, w1)
        tcg.sort()
        tc[g] = tcg.sum()
        beta = _lr_fit(xt, yt, wt)
        #model[g] = beta
        yh_ = _lr_predict(xh, beta)
        ho[g] = _lr_rss(yh, yh_, wh)
    MAX_VALID = 128
    tcfb = np.empty(MAX_VALID, dtype=tc.dtype) + nan32
    hofb = np.empty(MAX_VALID, dtype=tc.dtype) + nan32
    for i in range(MAX_VALID):
        temp = tc * (mvalid >= i) / (repeatf // f) + mbase * (mvalid < i)
        temp.sort()
        tcfb[i] = temp.sum()
        temp = ho * (mvalid >= i) + mbase * (mvalid < i)
        temp.sort()
        hofb[i] = temp.sum()
    hoi, hobest = hofb.argmin(), hofb.min()
    tci, tcbest = tcfb.argmin(), tcfb.min()
    return (tcbest, hobest), (tci, hoi)

In [4]:
@jit(nopython=True) # no fall back
def _alg_evaluate_canon(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, mvalid, mbase, k, subk, f):
    tc = np.empty(m, dtype=Yt.dtype) + nan32
    ho = np.empty(m, dtype=Yh.dtype) + nan32
    #model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    repeatf = X0.shape[0]
    for g in range(m):
        X0g, Y0g, W0g, X1g, Y1g, W1g, xt, yt, wt, xh, yh, wh = (
            X0[:, :, g::m][:, :, subk], Y0[:, :, g], W0[:, :, g],
            X1[:, :, g::m][:, :, subk], Y1[:, :, g], W1[:, :, g],
            Xt[:, g::m][:, subk], Yt[:, g], Wt[:, g],
            Xh[:, g::m][:, subk], Yh[:, g], Wh[:, g])
        tcg = np.empty(repeatf, dtype=Y0g.dtype) + nan32
        for i in range(repeatf):
            x0, y0, w0, x1, y1, w1 = X0g[i], Y0g[i], W0g[i], X1g[i], Y1g[i], W1g[i]
            beta = _lr_fit(x0, y0, w0)
            y1_ = _lr_predict(x1, beta)
            tcg[i] = _lr_rss(y1, y1_, w1)
        tcg.sort()
        tc[g] = tcg.sum()
        beta = _lr_fit(xt, yt, wt)
        #model[g] = beta
        yh_ = _lr_predict(xh, beta)
        ho[g] = _lr_rss(yh, yh_, wh)
    tc.sort()
    ho.sort()
    return (tc.sum() / (repeatf // f), ho.sum()), 'None'

In [ ]:
@jit(nopython=True)
def _alg_finalize_expfb(Xt, Yt, Wt, Xh, m, mvalid, k, subk, val, exp):
    model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    Yh_ = np.empty((Xh.shape[0], m), dtype=Xt.dtype) + nan32
    for g in range(m): # loop for each stock
        xt, yt, wt, xh = (
            Xt[:, g::m][:, subk], Yt[:, g], Wt[:, g],
            Xh[:, g::m][:, subk])
        beta = _lr_fit(xt, yt, wt)
        model[g] = beta
        yh_ = _lr_predict(xh, beta)
        if val >= mvalid[g] - subk.shape[0]:
            Yh_[:, g] = 0
        else:
            if exp == 0:
                Yh_[:, g] = yh_
            else:
                Yh_[:, g] = _exp_weigh(yh_, (mvalid[g] - subk.shape[0] - val) / exp)
    return model, Yh_

In [ ]:
@jit(nopython=True) # no fall back
def _alg_finalize_canon(Xt, Yt, Wt, Xh, m, mvalid, k, subk):
    model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    Yh_ = np.empty((Xh.shape[0], m), dtype=Xt.dtype) + nan32
    for g in range(m):
        xt, yt, wt, xh = (
            Xt[:, g::m][:, subk], Yt[:, g], Wt[:, g],
            Xh[:, g::m][:, subk])
        beta = _lr_fit(xt, yt, wt)
        model[g] = beta
        Yh_[:, g] = _lr_predict(xh, beta)
    return model, Yh_

In [ ]:
@jit(nopython=True)
def _alg_alltrain_expfb(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, mvalid, mbase, k, subk, f, _val, _exp):
    Mval, Mexp = len(_val), len(_exp)
    tc = np.zeros((m, Mval, Mexp), dtype=Yt.dtype) # zeros because we need to add to this for each fold
    #model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    repeatf = X0.shape[0]
    for g in range(m): # loop for each stock
        X0g, Y0g, W0g, X1g, Y1g, W1g = (
            X0[:, :, g::m][:, :, subk], Y0[:, :, g], W0[:, :, g],
            X1[:, :, g::m][:, :, subk], Y1[:, :, g], W1[:, :, g])
        ############## TRCV dataset
        for p in range(repeatf): # loop for each cv-fold
            x0, y0, w0, x1, y1, w1 = X0g[p], Y0g[p], W0g[p], X1g[p], Y1g[p], W1g[p]
            beta = _lr_fit(x0, y0, w0)
            y1_ = _lr_predict(x1, beta)
            # search over the exp fallback
            thei = Mval
            for i in range(Mval): # i + 1 = min num of obs in stock - ddof to use lin model
                if _val[i] >= mvalid[g] - subk.shape[0]:
                    thei = i
                    break
                tc[g, i, 0] += _lr_rss(y1, y1_, w1) # when j == 0 just use the linear model
                for j in range(1, Mexp): # j denotes exponential weighting argument
                    y1_fb = _exp_weigh(y1_, (mvalid[g] - subk.shape[0] - _val[i]) / _exp[j])
                    tc[g, i, j] += _lr_rss(y1, y1_fb, w1)
        # outside the cv-fold loop
        tc[g] /= repeatf // f # divide by number of times trcv set was repeated over
        tc[g, thei:] = mbase[g] # fill in the mean model prediction errors for when mvalid[g] < i
        #%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%#%
    tcfb = tc.sum(axis=0)
    tcij, tcbest = tcfb.argmin(), tcfb.min()
    return (tcbest, 0), (_val[tcij // Mexp], _exp[tcij % Mexp])

In [ ]:
@jit(nopython=True) # no fall back
def _alg_alltrain_canon(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, mvalid, mbase, k, subk, f):
    tc = np.empty(m, dtype=Yt.dtype) + nan32
    #model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    repeatf = X0.shape[0]
    for g in range(m):
        X0g, Y0g, W0g, X1g, Y1g, W1g = (
            X0[:, :, g::m][:, :, subk], Y0[:, :, g], W0[:, :, g],
            X1[:, :, g::m][:, :, subk], Y1[:, :, g], W1[:, :, g])
        tcg = np.empty(repeatf, dtype=Y0g.dtype) + nan32
        for i in range(repeatf):
            x0, y0, w0, x1, y1, w1 = X0g[i], Y0g[i], W0g[i], X1g[i], Y1g[i], W1g[i]
            beta = _lr_fit(x0, y0, w0)
            y1_ = _lr_predict(x1, beta)
            tcg[i] = _lr_rss(y1, y1_, w1)
        tcg.sort()
        tc[g] = tcg.sum()
    tc.sort()
    return (tc.sum() / (repeatf // f), 0), ()

In [5]:
@jit(nopython=True) #deprecated
def _alg_evaluate_2(X0, Y0, W0, X1, Y1, W1, Xt, Yt, Wt, Xh, Yh, Wh, m, k, subk):
    tc = np.empty(m, dtype=Yt.dtype) + nan32
    ho = np.empty(m, dtype=Yh.dtype) + nan32
    #model = np.empty((m, subk.shape[0]), dtype=Xt.dtype) + nan32
    repeat = X0.shape[0]
    for g in range(m):
        X0g, Y0g, W0g, X1g, Y1g, W1g, xt, yt, wt, xh, yh, wh = (
            X0[:, :, g::m][:, :, subk], Y0[:, :, g], W0[:, :, g],
            X1[:, :, g::m][:, :, subk], Y1[:, :, g], W1[:, :, g],
            Xt[:, g::m][:, subk], Yt[:, g], Wt[:, g],
            Xh[:, g::m][:, subk], Yh[:, g], Wh[:, g])
        tcg = np.empty(repeat * 2, dtype=Y0g.dtype) + nan32
        for i in range(repeat):
            x0, y0, w0, x1, y1, w1 = X0g[i], Y0g[i], W0g[i], X1g[i], Y1g[i], W1g[i]
            beta0, beta1 = _lr_fit(x0, y0, w0), _lr_fit(x1, y1, w1)
            y1_, y0_ = _lr_predict(x1, beta0), _lr_predict(x0, beta1)
            tcg[2 * i], tcg[2 * i + 1] = _lr_rss(y1, y1_, w1), _lr_rss(y0, y0_, w0)
        tcg.sort()
        tc[g] = tcg.sum()
        beta = _lr_fit(xt, yt, wt)
        #model[g] = beta
        yh_ = _lr_predict(xh, beta)
        ho[g] = _lr_rss(yh, yh_, wh)
    tc.sort()
    ho.sort()
    return (tc.sum() / 2, ho.sum())#, model

In [6]:
@jit(nopython=True)
def _lr_add_intercept(x): # NOT USED
    return np.concatenate((np.ones((x.shape[0], 1)), x), axis=1)

@jit(nopython=True)
def _lr_fit(x, y, w):
    wsqrt = w ** 0.5
    thing = np.linalg.lstsq(np.expand_dims(wsqrt, 1) * x, wsqrt * y)
    return thing[0]

@jit(nopython=True)
def _lr_predict(x, beta):
    return x @ beta.astype(x.dtype)

@jit(nopython=True)
def _lr_rss(y, y_, w):
    res = (y_ - y) ** 2 * w
    res.sort()
    return res.sum()

In [16]:
if __name__ == '__main__':
    o = O()
    def_W(o)
    def_Y(o)
    def_X(o)
    from models.split import split
    s = split2(o.X)
    ta = TheAlg(o.Xa, o.Ya, o.Wa, s)
    ta._setup()
    ta.alg1()

1.0 1.0 0
[] ()

<13> 0
1.0288687417628994 0.9984496220555278 1
[x4] (8,)

<12> 0
1.6107278811722803 1.0470519319703233 2
[x4, x5] (8, 9)

<11> 0
4.280990805481705 1.4171353539242328 3
[x2, x4, x5] (2, 8, 9)

<10> 0
4.841844442439121 2.036917539934384 4
[x2, x4, x5, x6] (2, 8, 9, 10)

<9> 0
5.318702137596149 5.774650711803322 5
[x0, x2, x4, x5, x6] (0, 2, 8, 9, 10)

